In [ ]:
from google.colab import drive

drive.mount("/content/gdrive/")

In [ ]:
cd /content/gdrive/MyDrive/grid-traversal/

In [1]:
from maze import Maze
from cell import *

In [2]:
def shortest_path(maze:Maze, start_cell:Cell, stop_cell:Cell, parents:dict):
    '''
    Returns the shortest path from a child:parent map
    '''
    grid = maze.grid
    path = []
    prev = stop_cell
    while prev != start_cell:
        path.append(grid[prev.row][prev.col])
        prev = parents[(prev.row, prev.col)]
    path.append(grid[start_cell.row][start_cell.col])
    path.reverse()
    return path

Breadth First Search

In [3]:
def breadth_first_search(maze:Maze, start_cell:Cell, stop_cell:Cell):
    '''
    Traverses through all neighbours at the current depth by using a FIFO queue/frontier
        maze : The Maze object to be traversed is a n*k matrix containing Cell objects
        start : A tuple of length 2 in the form of (row, col) that is the origin state
        stop : A typle of length 2 in the form of (row, col) that is the goal state
    '''
    grid:list[list] = maze.grid

    # Current state initialized at the starting state
    curr_cell:Cell = start_cell

    # Breadth first search uses a first-in-first-out (FIFO) stack where the oldest
    # element is popped from the queue
    frontier:list[Cell] = [curr_cell]

    # States that have already been visited should not be added to the queue again
    explored:list[Cell] = []

    # Map of traversed states and their immediate parent state for backtracking.
    # This is used to determine the shortest path by following a sequence of parents
    # back to the origin
    parents:dict[Cell] = {}

    # Continue while the queue contains traversable states
    while frontier:
        # Pop the oldest state in the queue and add it to the visited set
        curr_cell = frontier.pop(0)
        explored.append(curr_cell)

        # Once the current state has reached the goal state, end the process
        if curr_cell == stop_cell:
            return explored, parents
        
        # Add neighbours/children of the current state to the stack starting
        # from above in a clockwise order
        for row, col in [(-1,0), (0,1), (1,0), (0,-1)]:
            child_row, child_col = curr_cell.row+row, curr_cell.col+col

            # Check the child state is within the maze boundary
            if 0 <= child_row < len(grid) and 0 <= child_col < len(grid[0]):
                child_cell:Cell = grid[child_row][child_col]

                # Check the child cell is a traversable state in the maze.
                # Cells/states can be either a "path" or a "wall". Invalid states
                # block the path to the goal state
                if not child_cell.path:
                    continue
                
                # Add the neighbour/child to the queue if it has not been visited
                # Add the child and its parent for backtracking shortest path
                if (child_cell not in frontier) and (child_cell not in explored):
                    frontier.append(child_cell)
                    parents[(child_cell.row, child_cell.col)] = curr_cell

    # No solution
    return

Depth first search

In [4]:
def depth_first_search(maze:Maze, start_cell:Cell, stop_cell:Cell):
    '''
    Traverses to the end of a branch by using a FILO queue/frontier
        maze : The Maze object to be traversed is a n*k matrix containing Cell objects
        start : A tuple of length 2 in the form of (row, col) that is the origin state
        stop : A typle of length 2 in the form of (row, col) that is the goal state
    '''
    grid:list[list] = maze.grid

    # Current state initialized at the starting state
    curr_cell:Cell = start_cell

    # Depth first search uses a first-in-last-out (FILO) stack where the newest
    # element is popped from the queue
    frontier:list[Cell] = [curr_cell]

    # States that have already been visited should not be added to the queue again
    explored:list[Cell] = []

    # Map of traversed states and their immediate parent state for backtracking.
    # This is used to determine the shortest path by following a sequence of parents
    # back to the origin
    parents:dict[Cell] = {}

    # Continue while the queue contains traversable states
    while frontier:
        # Pop the newest state in the queue and add it to the visited set
        curr_cell = frontier.pop(-1)
        explored.append(curr_cell)

        # Once the current state has reached the goal state, end the process
        if curr_cell == stop_cell:
            return explored, parents
        
        # Add neighbours/children of the current state to the stack starting
        # from above in a clockwise order
        for row, col in [(-1,0), (0,1), (1,0), (0,-1)]:
            child_row, child_col = curr_cell.row+row, curr_cell.col+col

            # Check the child state is within the maze boundary
            if 0 <= child_row < len(grid) and 0 <= child_col < len(grid[0]):
                child_cell:Cell = grid[child_row][child_col]

                # Check the child cell is a traversable state in the maze.
                # Cells/states can be either a "path" or a "wall". Invalid states
                # block the path to the goal state
                if not child_cell.path:
                    continue
                
                # Add the neighbour/child to the queue if it has not been visited
                # Add the child and its parent for backtracking shortest path
                if (child_cell not in frontier) and (child_cell not in explored):
                    frontier.append(child_cell)
                    parents[(child_cell.row, child_cell.col)] = curr_cell

    # No solution
    return

A* search

In [5]:
import numpy as np

In [6]:
def heuristic(cell:Cell, stop_cell:Cell, metric:str="euclidean"):
        x_dist:int = abs(cell.row - stop_cell.row)
        y_dist:int = abs(cell.col - stop_cell.col)
        dist:float = None
        match metric:
            case "euclidean":
                sum_of_squares:int = (x_dist**2) + (y_dist**2)
                dist:float = np.sqrt(sum_of_squares)

                return dist
            case "manhattan":
                dist:float = sum((x_dist, y_dist))

        return dist

In [7]:
def a_star_search(maze:Maze, start_cell:tuple, stop_cell:tuple, metric:str="euclidean"):
    '''
    Traverses through neighbours that are closest to the goal state by using a priority frontier/queue.
        maze : The Maze object to be traversed is a n*k matrix containing Cell objects
        start : A tuple of length 2 in the form of (row, col) that is the origin state
        stop : A typle of length 2 in the form of (row, col) that is the goal state
    '''
    grid:list[list] = maze.grid

    # Current state initialized at the starting state
    curr_cell:Cell = start_cell

    # A* search uses a priority frontier/queue where the state with the lowest score
    # is popped from the stack to become the current state
    frontier:list[Cell] = [curr_cell]

    # States that have already been visited should not be added to the queue again
    explored:list[Cell] = []

    # Map of traversed states and their immediate parent state for backtracking.
    # This is used to determine the shortest path by following a sequence of parents
    # back to the origin
    parents:dict[Cell] = {(curr_cell.row, curr_cell.col):curr_cell}

    # A* score is calculated to order the frontier from lowest to highest
    # the first element (state with lowest score) is then popped to become the current state
    heuristic_cost = lambda cell:heuristic(
        cell=cell, stop_cell=stop_cell, metric=metric
    )
    current_cost:int = lambda cell:len(
        shortest_path(
            maze=maze, start_cell=start_cell, stop_cell=cell, parents=parents
        )
    )
    a_star_score = lambda cell:current_cost(cell=cell)+heuristic_cost(cell=cell)

    # Continue while the queue contains traversable states
    while frontier:
        frontier.sort(key=a_star_score)

        # Pop the newest state in the queue and add it to the visited set
        curr_cell = frontier.pop(0)
        explored.append(curr_cell)

        # Once the current state has reached the goal state, end the process
        if curr_cell == stop_cell:
            return explored, parents
        
        # Add neighbours/children of the current state to the stack starting
        # from above in a clockwise order
        for row, col in [(-1,0), (0,1), (1,0), (0,-1)]:
            child_row, child_col = curr_cell.row+row, curr_cell.col+col

            # Check the child state is within the maze boundary
            if 0 <= child_row < len(grid) and 0 <= child_col < len(grid[0]):
                child_cell:Cell = grid[child_row][child_col]

                # Check the child cell is a traversable state in the maze.
                # Cells/states can be either a "path" or a "wall". Invalid states
                # block the path to the goal state
                if not child_cell.path:
                    continue
                
                # Add the neighbour/child to the queue if it has not been visited
                # Add the child and its parent for backtracking shortest path
                if (child_cell not in frontier) and (child_cell not in explored):
                    frontier.append(child_cell)
                    parents[(child_cell.row, child_cell.col)] = curr_cell

    # No solution
    return

Evaluation

In [8]:
import tkinter as tk

In [40]:
def visualize(title:str, maze:Maze, start_cell:Cell, stop_cell:Cell, explored:list[Cell], cell_path:list[Cell]):
    window = tk.Tk()
    window.title(title)
    canvas = tk.Canvas(
        master=window, width=maze.width, height=maze.height
    )
    canvas.pack()

    for cell in explored:
        cell.colour = colour_set["explored"]
        start_cell.colour = colour_set["start"]
        stop_cell.colour = colour_set["stop"]
        maze.draw_grid(canvas=canvas)
        window.update()

    for cell in cell_path:
        cell.colour = colour_set["solution"]
        start_cell.colour = colour_set["start"]
        stop_cell.colour = colour_set["stop"]
        maze.draw_grid(canvas=canvas)
        window.update()

    window.mainloop()

Breadth First Search Evaluation

In [41]:
# Load maze for breadth first search traversal
bfs_maze = Maze(file_name="maze.csv")

# Run a search on each of the pathfinding algorithms and compare the results
start_cell:Cell = bfs_maze.grid[0][0]
stop_cell:Cell = bfs_maze.grid[19][19]

explored, parents = breadth_first_search(
    maze=bfs_maze, start_cell=start_cell, stop_cell=stop_cell
)
cell_path = shortest_path(maze=bfs_maze, start_cell=start_cell, stop_cell=stop_cell, parents=parents)
coord_path = [(cell.row, cell.col) for cell in cell_path]

print(coord_path)
print("Shortest path:", len(coord_path))
print("Explored states:", len(explored))

[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4), (4, 5), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (6, 11), (7, 11), (7, 12), (7, 13), (8, 13), (9, 13), (10, 13), (11, 13), (12, 13), (13, 13), (13, 14), (14, 14), (14, 15), (15, 15), (16, 15), (17, 15), (18, 15), (19, 15), (19, 16), (19, 17), (19, 18), (19, 19)]
Shortest path: 39
Explored states: 273


In [42]:
# tkinter visualization of breadth first search (incompatible with google colab)
visualize(
    title="breadth first search",
    maze=bfs_maze, 
    start_cell=start_cell, 
    stop_cell=stop_cell, 
    explored=explored, 
    cell_path=cell_path
)

Depth First Search Evaluation

In [43]:
# Load maze for depth first search traversal
dfs_maze = Maze(file_name="maze.csv")

# Run a search on each of the pathfinding algorithms and compare the results
start_cell:Cell = dfs_maze.grid[0][0]
stop_cell:Cell = dfs_maze.grid[19][19]

explored, parents = depth_first_search(
    maze=dfs_maze, start_cell=start_cell, stop_cell=stop_cell
)
cell_path = shortest_path(maze=dfs_maze, start_cell=start_cell, stop_cell=stop_cell, parents=parents)
coord_path = [(cell.row, cell.col) for cell in cell_path]

print(coord_path)
print("Shortest path:", len(coord_path))
print("Explored states:", len(explored))

[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (4, 1), (5, 1), (5, 0), (6, 0), (7, 0), (7, 1), (8, 1), (9, 1), (9, 0), (10, 0), (11, 0), (12, 0), (12, 1), (12, 2), (12, 3), (12, 4), (11, 4), (11, 5), (11, 6), (11, 7), (10, 7), (9, 7), (9, 6), (9, 5), (9, 4), (9, 3), (8, 3), (7, 3), (7, 4), (7, 5), (6, 5), (6, 6), (6, 7), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (8, 11), (8, 12), (8, 13), (9, 13), (10, 13), (11, 13), (11, 12), (12, 12), (13, 12), (14, 12), (15, 12), (15, 11), (15, 10), (15, 9), (15, 8), (14, 8), (14, 7), (14, 6), (15, 6), (16, 6), (17, 6), (17, 5), (17, 4), (18, 4), (19, 4), (19, 5), (19, 6), (19, 7), (19, 8), (18, 8), (18, 9), (18, 10), (19, 10), (19, 11), (19, 12), (19, 13), (19, 14), (19, 15), (19, 16), (19, 17), (19, 18), (19, 19)]
Shortest path: 85
Explored states: 122


In [46]:
# tkinter visualization of depth first search (incompatible with google colab)
visualize(
    title="depth first search",
    maze=dfs_maze, 
    start_cell=start_cell, 
    stop_cell=stop_cell, 
    explored=explored, 
    cell_path=cell_path
)

A* Search using Euclidean Distance Evaluation

In [47]:
# Load maze for euclidean a* search traversal
euc_maze = Maze(file_name="maze.csv")

# Run a search on each of the pathfinding algorithms and compare the results
start_cell:Cell = euc_maze.grid[0][0]
stop_cell:Cell = euc_maze.grid[19][19]

explored, parents = a_star_search(
    maze=euc_maze, start_cell=start_cell, stop_cell=stop_cell, metric="euclidean"
)
cell_path = shortest_path(maze=euc_maze, start_cell=start_cell, stop_cell=stop_cell, parents=parents)
coord_path = [(cell.row, cell.col) for cell in cell_path]

print(coord_path)
print("Shortest path:", len(coord_path))
print("Explored states:", len(explored))

[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4), (4, 5), (5, 5), (5, 6), (6, 6), (6, 7), (7, 7), (7, 8), (7, 9), (7, 10), (7, 11), (8, 11), (8, 12), (8, 13), (9, 13), (10, 13), (11, 13), (12, 13), (13, 13), (13, 14), (14, 14), (14, 15), (15, 15), (16, 15), (17, 15), (18, 15), (19, 15), (19, 16), (19, 17), (19, 18), (19, 19)]
Shortest path: 39
Explored states: 204


In [48]:
# tkinter visualization of euclidean a* search (incompatible with google colab)
visualize(
    title="euclidean a* search",
    maze=euc_maze, 
    start_cell=start_cell, 
    stop_cell=stop_cell, 
    explored=explored, 
    cell_path=cell_path
)

A* Search using Manhattan Distance Evaluation

In [49]:
# Load maze for euclidean a* search traversal
man_maze = Maze(file_name="maze.csv")

# Run a search on each of the pathfinding algorithms and compare the results
start_cell:Cell = man_maze.grid[0][0]
stop_cell:Cell = man_maze.grid[19][19]

explored, parents = a_star_search(
    maze=man_maze, start_cell=start_cell, stop_cell=stop_cell, metric="manhattan"
)
cell_path = shortest_path(maze=man_maze, start_cell=start_cell, stop_cell=stop_cell, parents=parents)
coord_path = [(cell.row, cell.col) for cell in cell_path]

print(coord_path)
print("Shortest path:", len(coord_path))
print("Explored states:", len(explored))

[(0, 0), (0, 1), (0, 2), (1, 2), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4), (4, 5), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (6, 11), (7, 11), (7, 12), (7, 13), (8, 13), (9, 13), (10, 13), (11, 13), (12, 13), (13, 13), (13, 14), (14, 14), (14, 15), (15, 15), (16, 15), (17, 15), (18, 15), (19, 15), (19, 16), (19, 17), (19, 18), (19, 19)]
Shortest path: 39
Explored states: 164


In [50]:
# tkinter visualization of manhattan a* search (incompatible with google colab)
visualize(
    title="manhattan a* search",
    maze=man_maze, 
    start_cell=start_cell, 
    stop_cell=stop_cell, 
    explored=explored, 
    cell_path=cell_path
)